In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/My Drive/IMDB/IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
df = df.dropna()

In [6]:
x=df.drop('sentiment', axis=1)

In [7]:
Y=df['sentiment']

In [8]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [9]:
import nltk
import re
from nltk.corpus import stopwords

In [10]:
from bs4 import BeautifulSoup

In [11]:
voc_size=10000

In [12]:
messages=x.copy()
messages.reset_index(inplace=True)
messages['review'][1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [13]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [36]:
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus=[]

In [15]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    return text

messages['review'] = messages['review'].apply(denoise_text)

In [37]:
for i in range(0, len(messages)):
    texts= re.sub('[^a-zA-Z]', ' ', messages['review'] [i])
    texts= texts.lower()
    texts= texts.split()
    
    texts = [ps.stem(word) for word in texts if not word in stopwords.words('english')]
    texts = ' '.join(texts)
    corpus.append(texts)

In [44]:
len(corpus)

50000

In [46]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
len(onehot_repr)

50000

In [47]:
sent_length =150
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[8287 6013 5306 ... 3127 6269 4812]
 [   0    0    0 ... 2463 6715 1624]
 [   0    0    0 ... 2739 1587 2698]
 ...
 [   0    0    0 ... 9497  820 2087]
 [   0    0    0 ... 4759  985 1558]
 [   0    0    0 ... 7689 3733  944]]


In [39]:
embedded_docs[0]

array([8287, 6013, 5306, 3456, 3841, 2275, 6785, 6604, 8456, 2739,  253,
       8657, 6325, 2808,  101, 8657, 3095,  461, 2136, 8520, 5278, 2275,
       2185, 4925, 2472, 8456, 9048, 6013, 4317, 5886, 4825, 6197, 1624,
       3992, 9154, 9288, 3769, 5866, 3368, 2428, 3575, 3642, 6585, 4525,
       3620, 7392, 8008, 2068, 3225, 5131, 8923, 3368, 8453, 4724,  978,
       4950,  285, 4359, 2087, 8827, 2613, 3863, 9324, 7976,  586, 5331,
       3772, 2851, 8462, 4661, 3441, 1776, 3938, 1642, 8073, 8657, 3169,
       8460, 8000, 8657, 8281, 4192, 4909, 2483,  425, 3742, 1269, 4192,
       4554, 4192, 1088, 6013, 1558, 1646, 4040, 9381, 6180, 8187, 8287,
       9509, 4492, 3938,  550, 6356, 9390, 5700, 6013, 5030, 8924, 3225,
       6756, 3501, 2275, 2275, 2565, 7768, 1852, 5335, 5292, 9933, 9123,
       1503, 8854, 3441, 6715, 2364, 4020, 3426, 9933, 7090, 3642, 6514,
       3169, 8493, 3045, 3822, 3642, 5229, 6356, 6013, 4935, 2027, 4025,
       5233, 2599, 2715, 8854, 3127, 6269, 4812], d

In [49]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 150, 40)           400000    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 456,501
Trainable params: 456,501
Non-trainable params: 0
_________________________________________________________________
None


In [50]:
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 150, 40)           400000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 201       
Total params: 513,001
Trainable params: 513,001
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
len(embedded_docs),Y.shape

(50000, (50000,))

In [53]:
from sklearn.preprocessing import LabelEncoder
Y= LabelEncoder().fit_transform(Y)

In [54]:
import numpy as np
x=np.array(embedded_docs)
Y=np.array(Y)

In [55]:
x.shape, Y.shape

((50000, 150), (50000,))

In [56]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, Y, test_size=0.33, random_state=42)

In [57]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
524/524 [==============================] - 8s 16ms/step - loss: 0.4101 - accuracy: 0.8155 - val_loss: 0.3033 - val_accuracy: 0.8730
Epoch 2/10
524/524 [==============================] - 8s 15ms/step - loss: 0.2608 - accuracy: 0.8955 - val_loss: 0.3079 - val_accuracy: 0.8718
Epoch 3/10
524/524 [==============================] - 8s 15ms/step - loss: 0.2159 - accuracy: 0.9175 - val_loss: 0.3208 - val_accuracy: 0.8696
Epoch 4/10
524/524 [==============================] - 8s 15ms/step - loss: 0.1722 - accuracy: 0.9357 - val_loss: 0.3608 - val_accuracy: 0.8633
Epoch 5/10
524/524 [==============================] - 8s 15ms/step - loss: 0.1369 - accuracy: 0.9500 - val_loss: 0.4090 - val_accuracy: 0.8543
Epoch 6/10
524/524 [==============================] - 8s 15ms/step - loss: 0.1152 - accuracy: 0.9591 - val_loss: 0.4325 - val_accuracy: 0.8482
Epoch 7/10
524/524 [==============================] - 8s 15ms/step - loss: 0.0984 - accuracy: 0.9663 - val_loss: 0.4921 - val_accuracy: 0.8523

In [58]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
524/524 [==============================] - 13s 24ms/step - loss: 0.4004 - accuracy: 0.8120 - val_loss: 0.3601 - val_accuracy: 0.8634
Epoch 2/10
524/524 [==============================] - 12s 23ms/step - loss: 0.2635 - accuracy: 0.8956 - val_loss: 0.3312 - val_accuracy: 0.8711
Epoch 3/10
524/524 [==============================] - 12s 23ms/step - loss: 0.2157 - accuracy: 0.9163 - val_loss: 0.3274 - val_accuracy: 0.8660
Epoch 4/10
524/524 [==============================] - 12s 23ms/step - loss: 0.1798 - accuracy: 0.9319 - val_loss: 0.3648 - val_accuracy: 0.8636
Epoch 5/10
524/524 [==============================] - 12s 23ms/step - loss: 0.1504 - accuracy: 0.9433 - val_loss: 0.4318 - val_accuracy: 0.8610
Epoch 6/10
524/524 [==============================] - 12s 23ms/step - loss: 0.1347 - accuracy: 0.9509 - val_loss: 0.4386 - val_accuracy: 0.8567
Epoch 7/10
524/524 [==============================] - 12s 23ms/step - loss: 0.1115 - accuracy: 0.9603 - val_loss: 0.4572 - val_accuracy:

In [59]:
y_pred1=model1.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [60]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred1)

array([[7114, 1094],
       [1439, 6853]])

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred1)

0.8464848484848485

In [62]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred1))

              precision    recall  f1-score   support

           0       0.83      0.87      0.85      8208
           1       0.86      0.83      0.84      8292

    accuracy                           0.85     16500
   macro avg       0.85      0.85      0.85     16500
weighted avg       0.85      0.85      0.85     16500

